<a href="https://colab.research.google.com/github/imenebak/VisDrone_CC/blob/master/Test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

In [ ]:
import cv2
import h5py
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm as c
from keras.models import model_from_json
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Image 2 video**

In [ ]:
!unzip "/content/VisDrone2020-CC-test.zip"

In [ ]:
import cv2
import numpy as np
import glob

nom_dossier = '/content/VisDrone2020-CC-test/00112'
img_array = []
for i in range(1, 31):
  if i > 9 :
      nom = "000"
  else : 
      nom = "0000"
  #print(nom_dossier+'/'+nom+str(i)+".jpg")
  img = cv2.imread(nom_dossier+'/'+nom+str(i)+".jpg")
  height, width, layers = img.shape
  size = (width,height)
  #print(filename)
  img_array.append(img)
 
 
out = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

# **Video 2 images**

In [ ]:
!rm -r "/content/image" 
os.mkdir("/content/image")

In [ ]:
import cv2
vidcap = cv2.VideoCapture('/content/project.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("/content/image/image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 0.1 #//it will capture image in each 0.1 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

# **Crowd Counting** (images)

In [ ]:
!cp "/content/drive/My Drive/ValTrain/visdr2020/best_weights.h5" "/content/"
!cp "/content/drive/My Drive/ValTrain/visdr2020/Model.json" "/content/"

In [ ]:
def load_model():
    # Function to load and return neural network model 
    json_file = open('/content/Model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("/content/best_weights.h5")
    return loaded_model

def create_img(path):
    #Function to load,normalize and return image 
    print(path)
    im = Image.open(path).convert('RGB')
    width, height = im.size 
    newsize = (int(width * .5), int(height * .5))
    #im = im.resize(newsize)
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225


    im = np.expand_dims(im,axis  = 0)
    return im

In [ ]:
model = load_model()

In [ ]:
root = "/content/"

In [ ]:
def predict(path):
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    #Function to load image,predict heat map, generate count and return (count , image , heat map)
    model = load_model()
    image = create_img(path)
    start_time = time.time()
    ans = model.predict(image)
    print("--- %s seconds ---" % (time.time() - start_time))
    count = np.sum(ans)
    return count,image,ans

In [ ]:
import time, matplotlib

In [ ]:
path_sets = os.path.join(root,'image')

In [ ]:
img_paths_test = []
for img_path in glob.glob(os.path.join(path_sets, '*.jpg')):
        img_paths_test.append(str(img_path))
print("Test image importation .. Total images : ",len(img_paths_test))
end = len(img_paths_test)+1

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

Density Estimation

In [ ]:
!rm -r "/content/resultat_comptage"
!rm -r "/content/resultat_density"

os.mkdir("/content/resultat_comptage")
os.mkdir("/content/resultat_density")

In [ ]:
import pandas as pd

In [ ]:
def norm_image(img):
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img

In [ ]:
def opencv_plot(densmap_name):

    densmap = np.array(densmap_name, np.float32)

    densmap = cv2.resize(densmap, (int(densmap.shape[1]*8), int(densmap.shape[0]*8)))
    densmap = cv2.blur(densmap, (10, 10))
    
    densmap[densmap >1 ] = 1
    densmap = norm_image(densmap) * 255
    densmap = densmap.astype(np.uint8)
    
    zmar = cv2.applyColorMap(densmap, cv2.COLORMAP_JET)
    
    return zmar
    #cv2.imshow("im", im_color)

In [ ]:
name = []
y_true = []
y_pred = []
tim = []
from matplotlib import cm as CM
for im in img_paths_test:
  name.append(im)
  nom = im.replace("/content/image/", "")
  frame = cv2.imread(im)   #image originale
  w, h,_ = frame.shape
  print(frame.shape)
  img = create_img(im)     #image entrainement
  #print(img.shape)
  start_time = time.time()
  data = model.predict(img)   #density map
  tim.append(float(time.time() - start_time))
  num = np.sum(data)
  y_pred.append(num)
  #print(data.shape, img.shape)
  ground_truth = data
  data = data.reshape(data.shape[1], data.shape[2], data.shape[3])
  #print(data.shape, img.shape)
  data = opencv_plot(data)
  frame = cv2.resize(frame,(data.shape[1], data.shape[0]))
  #print(data.shape, frame.shape)
  data = cv2.addWeighted(data, 0.3, frame , 1 - 0.3, 0, data)
  #print(data.shape)
  #cv2_imshow(data)
  
  #print(float(time.time() - start_time))
  fontface = cv2.FONT_HERSHEY_DUPLEX
  fontscale = 1
  fontcolor = (255, 255, 255)
  fr = cv2.putText(frame, "Count : "+str(int(num)+1), (50,50), fontface, fontscale, fontcolor) 
  frr = cv2.putText(data, "Count : "+str(int(num)+1), (50,50), fontface, fontscale, fontcolor) 
  #plt.imshow(fr)
  #fr = cv2.putText(frame,"Hello World!!!", (100,5), cv2.CV_FONT_HERSHEY_SIMPLEX, 2, 255)
  #print(num)
  #cv2_imshow(fr)
  
  cv2.imwrite("/content/resultat_comptage/"+nom, fr)
  cv2.imwrite("/content/resultat_density/"+nom, data)
  
dat = pd.DataFrame({'name': name,'y_pred': y_pred,"temp-ex": tim})
dat.to_csv('/content/resultat.csv', sep=',')

# **Resultats2videos**

In [ ]:
import cv2
import numpy as np
import glob
 
nom_dossier = '/content/resultat_comptage'
img_array = []
for i in range(1, end):
  nom = "image"
  print(nom_dossier+'/'+nom+str(i)+".jpg")
  img = cv2.imread(nom_dossier+'/'+nom+str(i)+".jpg")
  height, width, layers = img.shape
  size = (width,height)
  #print(filename)
  img_array.append(img)
 
 
out = cv2.VideoWriter('resltat_comptage.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
import cv2
import numpy as np
import glob
 
nom_dossier = '/content/resultat_density'
img_array = []
for i in range(1, end):
  nom = "image"
  print(nom_dossier+'/'+nom+str(i)+".jpg")
  img = cv2.imread(nom_dossier+'/'+nom+str(i)+".jpg")
  height, width, layers = img.shape
  size = (width,height)
  #print(filename)
  img_array.append(img)
 
 
out = cv2.VideoWriter('resltat_density.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()